<a href="https://colab.research.google.com/github/vitorfls/faster-whisper/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout

import numpy as np
from pandas import read_csv

# load the dataset
df = pd.read_csv('mega.csv')
df.head()


In [62]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [99]:
# All our games
number_of_rows = df.values.shape[0]
number_of_rows


2670

In [69]:
# Amount of games we need to take into consideration for prediction
window_length = 7
window_length

# Balls counts
number_of_features = df.values.shape[1]
number_of_features

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [77]:
model = Sequential()

model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = False)))

model.add(Dense(59))

model.add(Dense(number_of_features))

from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])




In [ ]:
# Next, let’s train our LSTM model
model.fit(x=X, y=y, batch_size=100, epochs=300, verbose=2)


In [86]:
to_predict = df.tail(8)
to_predict

,A,B,C,D,E,F
2662,7,11,27,41,56,59
2663,12,15,17,30,40,52
2664,3,14,21,22,37,39
2665,5,25,29,30,43,47
2666,1,4,8,21,46,51
2667,1,27,30,41,46,57
2668,4,7,16,35,46,54
2669,21,24,33,41,48,56


In [93]:
prediction = df.tail(1)
prediction

to_predict = np.array(to_predict)
to_predict


array([[ 7, 11, 27, 41, 56, 59],
       [12, 15, 17, 30, 40, 52],
       [ 3, 14, 21, 22, 37, 39],
       [ 5, 25, 29, 30, 43, 47],
       [ 1,  4,  8, 21, 46, 51],
       [ 1, 27, 30, 41, 46, 57],
       [ 4,  7, 16, 35, 46, 54]])

In [98]:
scaled_to_predict = scaler.transform(to_predict)
scaled_to_predict

y_pred = model.predict(np.array([scaled_to_predict]))
print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 0s 36ms/step
The predicted numbers in the last lottery game are: [22 26 33 41 48 56]
